In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI

# Aufgabe 08: Minimax mit Alpha-Beta-Pruning, Memoisierung, Progressive Deepening und Singular Value Extension

Dieses Notebook erweitert den Minimax-Algorithmus um die Singular Value Extension. Hierbei wird die Suchtiefe im jeweiligen Teilbaum um 1 erhöht, wenn entweder eine Figur geschlagen wurde, der König im Schach steht oder eine Figurenumwandlung durchgeführt wurde. Die Erhöhung wird dabei allerdings nur bis zu einem definierten Tiefenlimit (`MAX_SVE_DEPTH`) durchgeführt da sie ansonsten zu viel Ressourcen benötigt. Die Singular Value Extension reduziert den sog. Horizont-Effekt, d.h. eine gute Einschätzung eines Boards am Tiefenlimit wobei es aber einen nächsten Zug gibt, mit welchem sich die Stellung stark verschlechtert.

In [ ]:
import chess
from typing import Any, Callable


class Exercise08AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization and progressive deepening and the quiescence search"""

    def __init__(self, max_depth: int = 10, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, Any] = {}
        assert (
            self.DEPTH < max_depth
        ), "Max depth needs to be greater than search depth!"
        self.MAX_SVE_DEPTH: int = max_depth

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache.clear()

Minimax Debug Funktion: Printed den Minimax Game Tree

In [ ]:
def debug_minimax(evaluate_minimax: Callable):
    """Prints the Minimax Game Tree."""

    def minimax_debug(
        self,
        minimax: Callable,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        level: int = 0,
        alpha: int = -Exercise08AI.LIMIT,
        beta: int = Exercise08AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        color = "white" if board.turn else "black"
        ws = (level + 1) * "    "
        key = self.get_key(board)
        if key in self.cache:
            print(
                f"level: {level} {ws} mm> {color}; α: {alpha}; β: {beta}; limit: {limit}"
            )  # memoization
        else:
            print(
                f"level: {level} {ws} --> {color}; α: {alpha}; β: {beta}; limit: {limit}"
            )
        evaluation, best_move = evaluate_minimax(
            self, minimax, board, current_evaluation, limit, level, alpha, beta
        )
        print(
            f"level: {level} {ws} <-- {color}; ev: {evaluation}; best_move: {best_move}"
        )
        return evaluation, best_move

    return minimax_debug

Prüft ob der Zug ein Schlagzug, eine Umwandlung oder ein Schachzug ist und gibt True/False zurück. Push den Move dabei auf das Board.

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def is_quiet_move(self, board: chess.Board, move: chess.Move) -> bool:
        """Checks if the next move was an promotion, capture or check move. Pushes the given move on the board."""
        if move.promotion or board.piece_type_at(move.to_square):
            board.push(move)
            return False
        board.push(move)
        if board.is_check():
            return False
        return True

Neu:
- Die Tiefe wird nun anders interpretiert und hochgezählt statt herunter. Das ist intuitiver und vereinfacht den Programmcode auch deshalb, weil dann nicht mit negativen Zahlen gerechnet werden muss.  Aus diesem Grund ist die Abbruchbedingung hier dann auch `level == limit` und nicht mehr `level == 0`.
- Es gibt eine neue Abbruchbedingung wenn die maximale Tiefe erreicht ist. In diesem Fall soll die Kette von z.B. Schlagzügen nicht weiter untersucht werden. Da dieser Pfad dann unbekannt ist, soll er die jeweils schlechteste Wertung bekommen, damit er in nahezu keinem Fall gewählt wird. Diskussion: Alternativ könnte man auch das Board evaluieren und die Wertung zurückgeben, damit steigt aber wieder das Risiko eine Horizont-Effekts.


In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def minimax_early_abort(
        self, board: chess.Board, level: int, limit: int, current_evaluation: int
    ) -> int | None:
        """Returns an evaluation iff the minimax has an early exit condition. Returns None otherwise."""
        is_checkmate = board.is_checkmate()
        if is_checkmate and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - level
            return evaluation

        elif is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + level
            return evaluation

        elif (
            board.is_insufficient_material()
            or not board.legal_moves
            or board.is_fifty_moves()
        ):
            # Game is a draw
            return 0

        # Recursion abort case
        if level == limit:
            return current_evaluation

        # Maximum level reached
        if level == self.MAX_SVE_DEPTH:
            # Avoid this potentially bad path
            if board.turn:
                return self.LIMIT - level
            else:
                return -self.LIMIT + level

        return None

Neu:
- Die Funtkion Minimax wurde in die Funktionen maxValue, minValue und evaluate_minimax analog zum Skript aufgeteilt (im Skript heißt die nur `evaluate`, die ist bei uns aber schon belegt).
- minValue und maxValue anthalten jeweils den ihren Zweig aus der ehemaligen minimax-Funktion
- evaluate_minimax übernimmt nun die Memoisierung (kein Decorator mehr)
- Neuer Parameter `limit` (Suchtiefe von `get_next_middle_game_move`), der Parameter `real_depth` wird nicht mehr benötigt, diesen Wert hat nun der Parameter `depth`

In [ ]:
import heapq


class Exercise08AI(Exercise08AI):  # type: ignore
    def maxValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given limit using minimax algorithm"""
        early_abort_evaluation = self.minimax_early_abort(
            board, level, limit, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += level
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # White to play (positive numbers are good)
        for i, move in enumerate(board.legal_moves):
            key = self.get_key(board, limit - level - 1)
            old_eval = self.cache.get(key, (None, i))[1]
            heapq.heappush(moves, (-old_eval, i, move))
        maxEvaluation = alpha
        while moves:
            move = heapq.heappop(moves)[2]
            if level + 1 < limit:
                new_limit = limit
                board.push(move)
            elif self.is_quiet_move(board, move):
                new_limit = limit
            else:
                new_limit = limit + 1
            evaluation, _, = self.evaluate_minimax(
                self.minValue,
                board,
                self.evaluate(board, current_evaluation),
                new_limit,
                level + 1,
                maxEvaluation,
                beta,
            )
            board.pop()
            if evaluation >= beta:
                return evaluation, move
            if evaluation > maxEvaluation:
                best_move = move
                maxEvaluation = evaluation
        return maxEvaluation, best_move

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def minValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given limit using minimax algorithm"""
        early_abort_evaluation = self.minimax_early_abort(
            board, level, limit, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += level
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # Black to play (negative numbers are good)
        for i, move in enumerate(board.legal_moves):
            key = self.get_key(board, limit - level - 1)
            old_eval = self.cache.get(key, (None, i))[1]
            heapq.heappush(moves, (old_eval, i, move))
        minEvaluation = beta
        while moves:
            move = heapq.heappop(moves)[2]
            if level + 1 < limit:
                new_limit = limit
                board.push(move)
            elif self.is_quiet_move(board, move):
                new_limit = limit
            else:
                new_limit = limit + 1
            evaluation, _ = self.evaluate_minimax(
                self.maxValue,
                board,
                self.evaluate(board, current_evaluation),
                new_limit,
                level + 1,
                alpha,
                minEvaluation,
            )
            board.pop()
            if evaluation <= alpha:
                return evaluation, move
            if evaluation < minEvaluation:
                best_move = move
                minEvaluation = evaluation
        return minEvaluation, best_move

Neu:
- `get_key`: Der cache key enthält keine depth mehr
- `store_in_cache`: Die depth wird nun als Wert im Cache abgelegt (`limit - depth`)
  - Beispiel: Eine Funktionsaufruf mit depth 5 und limit 7 führt dazu, dass von dieser Stellung aus alle Positionen bis zum limit untersucht werden. Von dieser Stellung aus gesehen, ergibt das dann eine Tiefe von 2, welche dann auch im Cache gespeichert wird (7-5).
- `get_from_cache`: Hier wird neu überprüft ob die Tiefer größer ist als die Tiefe des Ergebnisses im Cache. Falls ja kann der Cache hier nicht verwendet werden und das Resultat muss erneut mit der höheren Tiefe berechnet werden.

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def get_key(self, board: chess.Board, depth: int) -> tuple:
        """Calculates a key that uniquely identifies a given board"""
        return (
            board.pawns,
            board.knights,
            board.bishops,
            board.rooks,
            board.queens,
            board.kings,
            board.occupied_co[chess.WHITE],
            board.occupied_co[chess.BLACK],
            board.turn,
            board.castling_rights,
            board.halfmove_clock if board.halfmove_clock > 42 else -42,
            depth,
        )

    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache[key] = ("≤", evaluation, move)
        elif evaluation < beta:
            self.cache[key] = ("=", evaluation, move)
        else:
            self.cache[key] = ("≥", evaluation, move)

    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        current_eval: int,
        limit: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(board, current_eval, limit, level, alpha, evaluation)
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(board, current_eval, limit, level, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(board, current_eval, limit, level, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(board, current_eval, limit, level, evaluation, beta)
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

Prinzipiell die gleiche Funktion wie vorher `memoize_minimax`.

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    # @debug_minimax
    def evaluate_minimax(
        self,
        minimax: Callable,
        board: chess.Board,
        current_evaluation: int,
        limit: int,
        level: int = 0,
        alpha: int = -Exercise08AI.LIMIT,
        beta: int = Exercise08AI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given limit using minimax algorithm"""
        key = self.get_key(board, limit - level)
        self.stats[-1]["cache_tries"] += 1
        self.stats[-1]["max_depth"] = max(level, self.stats[-1]["max_depth"])

        if key in self.cache:
            self.stats[-1]["cache_hits"] += 1
            return self.get_from_cache(
                minimax,
                key,
                board,
                current_evaluation,
                limit,
                level,
                alpha,
                beta,
            )
        result = minimax(board, current_evaluation, limit, level, alpha, beta)
        self.store_in_cache(key, result, alpha, beta)
        return result

Neu:
- Vor jeder Evaluierung wird geschaut ob der Spieler Weiß oder Schwarz ist. Je nachdem wird entweder die Referenz auf minValue oder maxValue übergeben. Ja, das ist eher hässlich und nicht so effizient, gerne noch anpassen. Prinzipiell steht ja schon nach dem ersten Zug die Farbe für das gesamt Spiel fest. Auch muss man das nicht innerhalb der Schleife prüfen. Wie gesagt, noch optimierungsfähig :-)

In [ ]:
import sys


class Exercise08AI(Exercise08AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        self.stats[-1]["cache_hits"] = 0
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["leaf_ctr"] = 0
        self.stats[-1]["max_depth"] = -1
        self.stats[-1]["avg_depth"] = -1
        self.stats[-1]["depth_sum"] = 0

        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for limit in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            if board.turn:
                future_evaluation, best_move = self.evaluate_minimax(
                    self.maxValue, board, current_evaluation, limit
                )
            else:
                future_evaluation, best_move = self.evaluate_minimax(
                    self.minValue, board, current_evaluation, limit
                )

        # Debugging fail safe
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(
            board, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation
        self.stats[-1]["avg_depth"] = self.stats[-1]["depth_sum"] / (
            self.stats[-1]["leaf_ctr"] or 1
        )
        del self.stats[-1]["depth_sum"]
        del self.stats[-1]["leaf_ctr"]
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        if board.is_irreversible(best_move):
            self.cache.clear()
            self.stats[-1]["cache_cleared"] = True
            self.cache: dict[tuple, Any] = {}
        else:
            self.stats[-1]["cache_cleared"] = False
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise03AI as Exercise03AITests
import Exercise04AI as Exercise04AITests

In [ ]:
# Create player and board
unit_test_player = Exercise08AI(player_name="Ex08AI", search_depth=2, max_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
def test_minimax(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    unit_test_player.stats[-1]["leaf_ctr"] = 0
    unit_test_player.stats[-1]["max_depth"] = 0
    unit_test_player.stats[-1]["depth_sum"] = 0
    f = unit_test_player.maxValue if board.turn else unit_test_player.minValue
    mm_evaluation, mm_move = unit_test_player.evaluate_minimax(
        f, board, current_evaluation=1240, limit=unit_test_player.DEPTH
    )
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == "d4c3", "Minimax move does not match expected value!"
    assert unit_test_player.cache != {}, "Cache is empty!"
    print(f"Elements in cache: {len(unit_test_player.cache)}")

In [ ]:
test_minimax(unit_test_player, board)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03AITests.test_next_move(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.